In [ ]:
!git clone https://github.com/mdeff/fma.git

In [1]:
1

1

In [ ]:
#!c1.4


In [ ]:
#!:bash
#pragma dataset init fma_small --size 20Gb
set -e

cd /home/jupyter/mnt/datasets/fma_small

wget https://os.unil.cloud.switch.ch/fma/fma_metadata.zip -P ./
echo "f0df49ffe5f2a6008d7dc83c6915b31835dfe733  ./fma_metadata.zip" | sha1sum -c -
7z x ./fma_metadata.zip -o./
rm -rf fma_metadata.zip

wget https://os.unil.cloud.switch.ch/fma/fma_small.zip -P ./
echo "ade154f733639d52e35e32f5593efe5be76c6d70  ./fma_small.zip" | sha1sum -c -
7z x ./fma_small.zip -o./
rm -rf fma_small.zip

In [ ]:
#!:bash
#pragma dataset init fma_medium --size 50Gb
set -e

cd /home/jupyter/mnt/datasets/fma_medium

wget https://os.unil.cloud.switch.ch/fma/fma_metadata.zip -P ./
echo "f0df49ffe5f2a6008d7dc83c6915b31835dfe733  ./fma_metadata.zip" | sha1sum -c -
7z x ./fma_metadata.zip -o./
rm -rf fma_metadata.zip

wget https://os.unil.cloud.switch.ch/fma/fma_medium.zip -P ./
echo "c67b69ea232021025fca9231fc1c7c1a063ab50b  fma_medium.zip"   | sha1sum -c -
7z x ./fma_medium.zip -o./
rm -rf fma_medium.zip

In [ ]:
import tensorflow
from tensorflow.python.client import device_lib

def get_available_gpus():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos]

get_available_gpus()

In [ ]:
#!g2.mig
#pragma async
get_available_gpus()

In [ ]:
%pip install python-dotenv kapre



In [ ]:
import librosa
import librosa.display

import fma.utils as fma
import IPython.display as ipd

PREFIX = "/home/jupyter/mnt/datasets/fma_small/"
AUDIO_DIR = PREFIX + "fma_small/"

In [ ]:
tracks = fma.load(PREFIX + 'fma_metadata/tracks.csv')
genres = fma.load(PREFIX + 'fma_metadata/genres.csv')

small = tracks[tracks['set', 'subset'] <= 'small']

In [ ]:
filename = fma.get_audio_path(AUDIO_DIR, 2)
x, sr = librosa.load(filename, sr=None, mono=True)
print('Duration: {:.2f}s, {} samples'.format(x.shape[-1] / sr, x.size))

best_sr = 5000
x = librosa.resample(x, sr, best_sr)

start, end = 7, 17
ipd.Audio(data=x[start*best_sr:end*best_sr], rate=best_sr)

In [ ]:
from joblib import Parallel, delayed 
import time

class ProgressParallel(Parallel):
    def __init__(self, use_tqdm=True, total=None, *args, **kwargs):
        self._use_tqdm = use_tqdm
        self._total = total
        super().__init__(*args, **kwargs)

    def __call__(self, *args, **kwargs):
        with tqdm(disable=not self._use_tqdm, total=self._total) as self._pbar:
            return Parallel.__call__(self, *args, **kwargs)

    def print_progress(self):
        if self._total is None:
            self._pbar.total = self.n_dispatched_tasks
        self._pbar.n = self.n_completed_tasks
        self._pbar.refresh()

In [14]:

import itertools
from tqdm.notebook import tqdm
import numpy as np
from joblib import Parallel, delayed 
from audioread.exceptions import NoBackendError
import warnings
import random
warnings.filterwarnings('ignore', message=".*PySoundFile.*")


SAMPLE_RATE = 4096
LENGTH_SECONDS = 1.0
LENGTH = int(round((LENGTH_SECONDS * SAMPLE_RATE)))

COUNT = 8000

def extract_audio(i, row):
    warnings.filterwarnings('ignore', message=".*PySoundFile.*")
    filename = fma.get_audio_path(AUDIO_DIR, i)
    try:
        data, sr = librosa.load(filename, sr=SAMPLE_RATE, mono=True, duration=LENGTH_SECONDS + 0.01)
        if len(data) < LENGTH:
            return None
        
        data = data[0: LENGTH]
        return data, row["track"]["genre_top"]
    except NoBackendError:
        return None

def get_dataset():
    tracks_to_parse = list(itertools.islice(small.iterrows(), COUNT))
    task = [delayed(extract_audio)(i, row) for i, row in tracks_to_parse]
    dataset = ProgressParallel(n_jobs=4, total=len(tracks_to_parse))(task)
    print("dataset", len(dataset))
    dataset = [i for i in dataset if i is not None]
    print("filtered", len(dataset))
    x, y = zip(*dataset)
    x, y = np.array(x), np.array(y)
    print("x, y", x.shape, y.shape)
    return x, y
    
def prepare_dataset():
    x, y = get_dataset()

    np.save("x", x)
    np.save("y", y)
    print("success")

prepare_dataset()

Execute error: Servant c1.4 not allocated: Allocation was interrupted

In [ ]:
tracks_to_parse = list(itertools.islice(small.iterrows(), COUNT))
a, b = extract_audio(*tracks_to_parse[2])
len(a)

In [ ]:
len(new_x)

In [ ]:
x = np.load("x.npy")
y = np.load("y.npy")

In [ ]:
x.shape, y.shape

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

def prepare_train_test(x, y):
    df = pd.DataFrame(data={'col1': y})
    y_cat = pd.get_dummies(df['col1']).to_numpy()
    return train_test_split(x, y_cat, test_size=0.2, random_state=42)



In [ ]:
X_train, X_test, y_train, y_test = prepare_train_test(np.load("x.npy"), np.load("y.npy"))

In [ ]:

Parallel(n_jobs=10)(delayed(time.sleep)(i ** 2) for i in tqdm(range(20)))

In [ ]:
len(x), len(y)

In [ ]:
np.sum(y_train, axis=0)

In [ ]:
np.sum(y_test, axis=0)

In [ ]:
#!g2.mig


In [ ]:
#!g2.mig
import kapre

from keras.models import Sequential
from keras.layers import Conv2D, BatchNormalization, ReLU, GlobalAveragePooling2D, Dense, Softmax, Activation
from kapre import STFT, Magnitude, MagnitudeToDecibel
from kapre.composed import get_melspectrogram_layer, get_log_frequency_spectrogram_layer
from tensorflow.keras import activations
from tensorflow.keras.optimizers import Adam
input_shape = (LENGTH, 1)

model = Sequential()
# model.add(STFT(n_fft=2048, win_length=2048, hop_length=1024,
#                window_name=None, pad_end=False,
#                input_data_format='channels_last', output_data_format='channels_last',
#                input_shape=input_shape))

model.add(STFT(n_fft=4096, win_length=4096, hop_length=2048,
               window_name="hann_window", pad_end=False,
               input_data_format='channels_last', output_data_format='channels_last',
               input_shape=input_shape))


model.add(Magnitude())
model.add(MagnitudeToDecibel())

# model.add(Dense(50))
model.add(Activation(activations.tanh))
model.add(Dense(50))
model.add(ReLU())
model.add(Dense(50))
# model.add(Conv2D(32, (3, 3), strides=(2, 2)))
model.add(BatchNormalization())
model.add(ReLU())
# model.add(GlobalAveragePooling2D())
model.add(Dense(8))
model.add(Softmax())

# Compile the model
model.compile(Adam(learning_rate=0.0001), 'categorical_crossentropy', 'categorical_accuracy')


model.fit(X_train, y_train, epochs=100, validation_data=(X_test, y_test))       

In [ ]:
#!g2.mig
y_pred = model.predict(X_test)
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test.argmax(axis=1), y_pred.argmax(axis=1))

In [19]:
#!g2.mig
def test():
    import numpy as np
    import librosa
    x = np.load("x.npy")
    print(librosa.stft(x[0]).shape)
test()

(1025, 9)


In [1]:
#!g2.mig

def get_x_stft(x):
    import numpy as np
    import librosa
    
    x = np.load("x.npy")
    x_stft = []
    for i in x[:]:
        x_stft.append(librosa.stft(i))

    x_stft = np.array(x_stft)
    
    return x_stft

def nnorm(x_stft):
    import numpy as np
    aa = np.ma.log2(np.abs(x_stft) + 1).filled(0)
    mmin = 0 #np.min(aa), 
    mmax = 10 # np.max(aa)
    mag = (aa - mmin) / (mmax - mmin)
    ang = (np.angle(x_stft) + np.pi) / 2 / np.pi
    
    return np.dstack((mag, ang))

def denorm(mag_ang):
    import numpy as np
    mag, ang = np.dsplit(mag_ang, 2)
    
    mmin = 0 
    mmax = 10
    
    aa = mag * (mmax - mmin) + mmin
    aa = np.exp2(aa) - 1
    ang2 = ang * 2 * np.pi - np.pi
    
    return aa * np.exp(1j*ang2)

def get_ddata(x_stft):
    import numpy as np
    mag_ang = nnorm(x_stft)
    mag, ang = np.dsplit(mag_ang, 2)
#     return mag[:, :, 0:2]
    return mag[:, :, 0:]
#     return np.dstack((mag[:, :, 0], ang[:, :, 0]))

# X_train, X_test, y_train, y_test = prepare_train_test(x_stft, y)

In [122]:
#!g2.mig

def test():
    import numpy as np
    print(get_ddata(get_x_stft(np.load("x.npy"))).shape  )
#     x_stft = get_x_stft(np.load("x.npy"))
#     mag_ang = nnorm(x_stft)
#     mag, ang = np.dsplit(mag_ang, 2)
#     print(mag.shape)
test()

(7996, 1025, 9)


In [ ]:
#!g2.mig
np.min(get_ddata(get_x_stft(np.load("x.npy")))), np.max(get_ddata(get_x_stft(np.load("x.npy")))), 

In [2]:
#!g2.mig
from tensorflow.keras.backend import clear_session
clear_session()

In [2]:
#!g2.mig
!nvidia-smi

Fri Sep 16 18:38:33 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.103.01   Driver Version: 470.103.01   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GRID A100X-1-5C     On   | 00000000:8C:00.0 Off |                   On |
| N/A   N/A    P0    N/A /  N/A |                  N/A |     N/A      Default |
|                               |                      |              Enabled |
+-------------------------------+----------------------+----------------------+

+-----------------------------------------------------------------------------+
| MIG d

In [3]:
#!g2.mig

def model_predict_mag(model, mag):
    import numpy as np
    pred_mag_1024 = model.predict(mag[:, :1024, :])
    mag_1025 = np.concatenate((pred_mag_1024, mag[:, 1024:1025, :]), axis=-2)
    return mag_1025

def getAutoencoder():
    from keras import Model
    class Autoencoder(Model):
        def __init__(self):
            super(Autoencoder, self).__init__()
            from keras.layers import Conv2D, Conv1D, BatchNormalization, ReLU, MaxPooling1D, UpSampling1D, Dense, Softmax, Reshape, Flatten, Input
            import tensorflow as tf
            
            input_shape = (1024, 9)
            self.encoder = tf.keras.Sequential([
                Input(input_shape),
                Conv1D(filters=10, kernel_size=5, activation='tanh', padding="same"),
                MaxPooling1D(pool_size = 2),
                Conv1D(filters=10, kernel_size=5, activation='tanh', padding="same"),
                MaxPooling1D(pool_size = 2),
                Conv1D(filters=10, kernel_size=5, activation='tanh', padding="same"),
                MaxPooling1D(pool_size = 2),
                Conv1D(filters=10, kernel_size=5, activation='tanh', padding="same"),
                MaxPooling1D(pool_size = 2),
                Conv1D(filters=1, kernel_size=5, activation='tanh', padding="same"),
                Dense(16, activation='tanh'),
            ])
            self.decoder = tf.keras.Sequential([
                Conv1D(filters=10, kernel_size=5, activation='tanh', padding="same"),
                UpSampling1D(size=2),
                Conv1D(filters=10, kernel_size=5, activation='tanh', padding="same"),
                UpSampling1D(size=2),
                Conv1D(filters=10, kernel_size=5, activation='tanh', padding="same"),
                UpSampling1D(size=2),
                Conv1D(filters=10, kernel_size=5, activation='tanh', padding="same"),
                UpSampling1D(size=2),
                Conv1D(filters=10, kernel_size=5, activation='tanh', padding="same"),
                Conv1D(filters=9, kernel_size=5, activation='tanh', padding="same"),
                Reshape( (1024, 9) ),
            ])

        def call(self, x):
            encoded = self.encoder(x)
            decoded = self.decoder(encoded)
            return decoded
        
        def save(self):
            self.encoder.save('encoder.keras')
            self.decoder.save('decoder.keras')
            
        def load(self):
            from keras.models import load_model
            self.encoder = load_model("encoder.keras")
            self.decoder = load_model("decoder.keras")
        
    return Autoencoder


def run():
    import numpy as np
    print("loading_data")
    mag_ang = get_ddata(get_x_stft(np.load("x.npy")))
    print("importing")
    from keras import Model
    from keras.models import Sequential
    # from kapre import InverseSTFT, STFT, Magnitude, MagnitudeToDecibel
    # from kapre.composed import get_melspectrogram_layer, get_log_frequency_spectrogram_layer
    from keras.backend import cast as Cast
    from tensorflow.keras.optimizers import Adam
    import tensorflow as tf
    print("preparing")

    

    
    normalizer = tf.keras.layers.Normalization(axis=-1)
    normalizer.adapt(mag_ang)
    
# #     model.add(Flatten())
# #     model.add(BatchNormalization())
# #     model.add(Dense(1025 * 1, activation='sigmoid'))


    autoencoder = getAutoencoder()()
    #learning_rate=0.05 learning_rate=0.001 binary_crossentropy
    autoencoder.compile(Adam(), 'mean_squared_error', 'mean_absolute_error')

    print("training")
    import datetime
    log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
    #
    def my_callback(*args, **kwargs):
        import numpy as np
        m1, a1 = np.dsplit(model.predict(mag_ang[:1, :, :]), 2)
        m2, a2 = np.dsplit(mag_ang[:1, :, :], 2)
        import matplotlib.pyplot as plt
        plt.plot(m1.reshape(1025)[:100])
        plt.plot(m2.reshape(1025)[:100])
        plt.show()
        
    def my_callback2(*args, **kwargs):
        import numpy as np
        m1 = model_predict_mag(autoencoder, mag_ang[:1, :, :])
        m2 = mag_ang[:1, :, :]
        import matplotlib.pyplot as plt
        plt.plot(m1[:, :, 0].reshape(1025)[:100])
        plt.plot(m2[:, :, 0].reshape(1025)[:100])
        plt.show()
        
    def scheduler(epoch):
        if epoch % 5 == 4:
            autoencoder.save()
        if epoch < 30:
            return 0.001
        elif epoch < 100:
            return 0.0005
        else:
            return 0.0001
        
    autoencoder.fit(mag_ang[:50, :1024, :], mag_ang[:50, :1024, :], epochs=600, 
            callbacks=[tensorboard_callback, tf.keras.callbacks.LambdaCallback(on_epoch_end=my_callback2),
                       tf.keras.callbacks.LearningRateScheduler(scheduler)], validation_split=0.2)
    autoencoder.save()
    
    
def run_proc(task):
    import multiprocessing
    print("starting proc")
    p = multiprocessing.Process(target=task)
    try:
        p.start()
        p.join()
    except KeyboardInterrupt:
        print("terminating proccess")
        p.terminate()
        from time import sleep
        sleep(1)
        print("killing")
        p.kill()
    
# run_proc(run)

In [ ]:
#!g2.mig


In [ ]:
#!g2.mig


In [7]:
#!g2.mig
def run():
    import numpy as np
    print("reading and calculating stft")
    mag_ang = nnorm(get_x_stft(np.load("x.npy")))
    mag, ang = np.dsplit(mag_ang, 2)
    
    print("preparing y")
    import pandas as pd
    y = np.load("y.npy")
    df = pd.DataFrame(data={'col1': y})
    y_cat = pd.get_dummies(df['col1']).to_numpy()
    
    print("proccessing")
    mag_encoded = mag[:, :1024, :]
    
    from keras.layers import Conv2D, Conv1D, BatchNormalization, ReLU, MaxPooling1D, UpSampling1D, Dense, Softmax, Reshape, Flatten, Input, Dropout
    import tensorflow as tf
    from tensorflow.keras.optimizers import Adam
    
    model = tf.keras.Sequential([
               Input((1024, 9)),
                Conv1D(filters=10, kernel_size=5, activation='tanh', padding="same"),
                BatchNormalization(),
                MaxPooling1D(pool_size = 2),
                Conv1D(filters=10, kernel_size=5, activation='tanh', padding="same"),
                BatchNormalization(),
                MaxPooling1D(pool_size = 2),
                Conv1D(filters=10, kernel_size=5, activation='tanh', padding="same"),
                BatchNormalization(),
                MaxPooling1D(pool_size = 2),
                Conv1D(filters=10, kernel_size=5, activation='tanh', padding="same"),
                BatchNormalization(),
                MaxPooling1D(pool_size = 2),
                Flatten(),
                Dense(512, activation='relu'),
                Dropout(0.6),
                BatchNormalization(),
        
                Dense(512, activation='relu'),
                Dropout(0.6),
                BatchNormalization(),
        
                Dense(512, activation='relu'),
                Dropout(0.6),
                BatchNormalization(),
        
                Dense(8, activation='softmax'),
            ])
#     model = tf.keras.Sequential([
#             Input((64, 16)),
#             Flatten(),
#             Dense(256, activation='tanh'),
#             Dense(256, activation='sigmoid'),
#             Dense(256, activation='relu'),
#             Dense(8, activation='softmax'),
#         ])
    
    def scheduler(epoch):
        if epoch % 5 == 4:
            model.save("model.keras")
        return 0.001
        if epoch < 30:
            return 0.001
        elif epoch < 100:
            return 0.0005
        else:
            return 0.0001
    print(mag_encoded.shape, y_cat.shape)
    model.compile(Adam(), 'categorical_crossentropy', 'categorical_accuracy')
    model.fit(mag_encoded, y_cat, epochs=100, 
            callbacks=[tf.keras.callbacks.LearningRateScheduler(scheduler)], validation_split=0.2, verbose=2)

run_proc(run)

starting proc
reading and calculating stft
preparing y
proccessing


2022-09-16 18:44:22.836232: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-09-16 18:44:23.269180: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 2629 MB memory:  -> device: 0, name: GRID A100X-1-5C MIG 1g.5gb, pci bus id: 0000:8c:00.0, compute capability: 8.0


(7996, 1024, 9) (7996, 8)


2022-09-16 18:44:23.763567: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/100


2022-09-16 18:44:25.902550: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8101
2022-09-16 18:44:27.037299: I tensorflow/stream_executor/cuda/cuda_blas.cc:1760] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


200/200 - 5s - loss: 2.5315 - categorical_accuracy: 0.1595 - val_loss: 2.1721 - val_categorical_accuracy: 0.1781
Epoch 2/100
200/200 - 2s - loss: 2.2809 - categorical_accuracy: 0.1704 - val_loss: 1.8721 - val_categorical_accuracy: 0.2988
Epoch 3/100
200/200 - 1s - loss: 2.1256 - categorical_accuracy: 0.1887 - val_loss: 1.8476 - val_categorical_accuracy: 0.3094
Epoch 4/100
200/200 - 1s - loss: 2.0428 - categorical_accuracy: 0.2073 - val_loss: 1.8845 - val_categorical_accuracy: 0.2775
Epoch 5/100
200/200 - 2s - loss: 1.9531 - categorical_accuracy: 0.2481 - val_loss: 1.9078 - val_categorical_accuracy: 0.2506
Epoch 6/100
200/200 - 1s - loss: 1.9163 - categorical_accuracy: 0.2569 - val_loss: 2.0311 - val_categorical_accuracy: 0.2606
Epoch 7/100
200/200 - 1s - loss: 1.8613 - categorical_accuracy: 0.2942 - val_loss: 1.7454 - val_categorical_accuracy: 0.3200
Epoch 8/100
200/200 - 1s - loss: 1.8491 - categorical_accuracy: 0.2971 - val_loss: 1.7442 - val_categorical_accuracy: 0.3381
Epoch 9/100


In [35]:
#!g2.mig
def test():
    import numpy as np
    print("processing x")
    mag_ang = nnorm(get_x_stft(np.load("x.npy")))
    mag, ang = np.dsplit(mag_ang, 2)
    
    print("  autoencoder")
    autoencoder = getAutoencoder()()
    autoencoder.load()
    autoencoder.encoder.compile()
    mag_encoded = autoencoder.encoder.predict(mag[:, :1024, :])
    
    print("processing y")
    import pandas as pd
    y = np.load("y.npy")
    df = pd.DataFrame(data={'col1': y})
    y_cat = pd.get_dummies(df['col1']).to_numpy()
    
    print("load model")
    from keras.models import load_model
    model = load_model("model.keras")
    
    print("confusion matrix")
    y_pred = model.predict(mag_encoded)
    from sklearn.metrics import confusion_matrix
    print(y_pred.shape)
    print(y_pred[:3])
    print(confusion_matrix(y_cat.argmax(axis=1), y_pred.argmax(axis=1)))

run_proc(test)

2022-09-16 16:26:34.905193: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-09-16 16:26:35.334946: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 2629 MB memory:  -> device: 0, name: GRID A100X-1-5C MIG 1g.5gb, pci bus id: 0000:8c:00.0, compute capability: 8.0
2022-09-16 16:26:36.054330: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2022-09-16 16:26:36.743816: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8101
2022-09-16 16:26:37.933607: I tensorflow/stream_executor/cuda/cuda_blas.cc:1760] TensorFloat-32 will be used for the matrix multipli

starting proc
processing x
  autoencoder
processing y
load model
confusion matrix
(7996, 8)
[[0.11667339 0.14175113 0.11960492 0.12665664 0.12259732 0.12679233
  0.13212846 0.11379588]
 [0.14226915 0.12479449 0.08894964 0.17497246 0.10102151 0.1280257
  0.1292968  0.11067025]
 [0.09513927 0.13377663 0.10467512 0.09310499 0.0640965  0.1157955
  0.12832224 0.26508972]]
[[  0 230  34 514  61  30   0 130]
 [  1 345  98 122 162 130   0 141]
 [  0 169 390  48 177 118   0  98]
 [  1 151  10 560  14  53   0 210]
 [  1 285 153  83 347  79   0  52]
 [  0 237  91 163  70 300   0 139]
 [  0 269  84 225  61 108   0 253]
 [  0 138  68 174  32  47   0 540]]


In [ ]:
#!g2.mig


In [ ]:
#!g2.mig
#!g2.mig
def run():
    import numpy as np
    print("loading autoencoder")
    autoencoder = getAutoencoder()()
    autoencoder.load()
    autoencoder.encoder.compile()
    print("reading and calculating stft")
    mag_ang = nnorm(get_x_stft(np.load("x.npy")))
    mag, ang = np.dsplit(mag_ang, 2)
    
    print("preparing y")
    import pandas as pd
    y = np.load("y.npy")
    df = pd.DataFrame(data={'col1': y})
    y_cat = pd.get_dummies(df['col1']).to_numpy()
    
    print("proccessing")
    mag_encoded = autoencoder.encoder.predict(mag[:, :1024, :])
    print(mag_encoded.shape)
    
    from keras.layers import Conv2D, Conv1D, BatchNormalization, ReLU, MaxPooling1D, UpSampling1D, Dense, Softmax, Reshape, Flatten, Input, Dropout
    import tensorflow as tf
    from tensorflow.keras.optimizers import Adam
    
    model = tf.keras.Sequential([
                Input((64, 16)),
                Flatten(),
                Dense(512, activation='tanh'),
                Dropout(0.8),
                Dense(512, activation='sigmoid'),
                Dropout(0.8),
                Dense(512, activation='relu'),
                Dropout(0.5),
                Dense(8, activation='softmax'),
            ])
#     model = tf.keras.Sequential([
#             Input((64, 16)),
#             Flatten(),
#             Dense(256, activation='tanh'),
#             Dense(256, activation='sigmoid'),
#             Dense(256, activation='relu'),
#             Dense(8, activation='softmax'),
#         ])
    
    def scheduler(epoch):
        if epoch % 5 == 4:
            model.save("model.keras")
        return 0.001
        if epoch < 30:
            return 0.001
        elif epoch < 100:
            return 0.0005
        else:
            return 0.0001
    print(mag_encoded.shape, y_cat.shape)
    model.compile(Adam(), 'categorical_crossentropy', 'categorical_accuracy')
    model.fit(mag_encoded, y_cat, epochs=100, 
            callbacks=[tf.keras.callbacks.LearningRateScheduler(scheduler)], validation_split=0.2, verbose=2)

run_proc(run)

In [ ]:
#!g2.mig
def test():
    import numpy as np
    import pandas as pd
    y = np.load("y.npy")
    df = pd.DataFrame(data={'col1': y})
    y_cat = pd.get_dummies(df['col1']).to_numpy()
    return y_cat.sum(axis=0)
test()

In [21]:
#!g2.mig
import numpy as np
a = np.array([[[1, 100], [2, 200], [2, 300], [4, 123], [5, 1234]], 
              [[5, 12345], [6, 51235], [7, 152], [8, 51235], [123, 42134253]]])
a.shape

(2, 5, 2)

In [25]:
#!g2.mig
np.concatenate((a[:, :3, :], a[:, 3:4, :]), axis=2)

ValueError: all the input array dimensions for the concatenation axis must match exactly, but along dimension 1, the array at index 0 has size 3 and the array at index 1 has size 1

In [4]:
#!g2.mig
1

1

In [ ]:
#!g2.mig


In [ ]:
#!g2.mig


In [ ]:
#!g2.mig


In [ ]:
#!g2.mig


In [ ]:
#!g2.mig


In [ ]:
#!g2.mig
x_stft[0].shape

In [ ]:
#!g2.mig
np.sum((aaa -bbb))

In [ ]:
#!g2.mig


In [ ]:
#!g2.mig


In [ ]:
#!g2.mig
mag_ang[0, :, :].shape

In [ ]:
#!g2.mig
mag_ang[0:, :, :].shape

In [ ]:
#!g2.mig
def test():
    from keras.models import load_model
    import numpy as np

    mag_ang = nnorm(get_x_stft(np.load("x.npy")))
    mag, ang = np.dsplit(mag_ang[0:1, :, :], 2)
    ma = np.dstack((mag[:, :, 2], ang[:, :, 2]))
    model = load_model('model.keras')
#     m1, a1 = np.dsplit(model.predict(ma), 2)
    m2, a2 = np.dsplit(ma, 2)
    m1, a1 = model.predict(m2), a2

    np.save("m1,a1,m2,a2", [m1.reshape(1025),a1.reshape(1025),m2.reshape(1025),a2.reshape(1025)])

def run():
    run_proc(test)
    m1,a1,m2,a2 = np.load("m1,a1,m2,a2.npy", allow_pickle=True)
    
    import matplotlib.pyplot as plt
    plt.plot(m1[:100])
    plt.plot(m2[:100])
    plt.show()
    plt.plot(a1[:100])
    plt.plot(a2[:100])
run()

In [ ]:
#!g2.mig


In [ ]:
#!g2.mig


In [75]:
#!g2.mig


def test():
    global x_stft_pred
    import numpy as np
    model = getAutoencoder()()
    model.load()
    
    x_stft_pred = []
    mag_ang = nnorm(get_x_stft(np.load("x.npy")))[1:2, :, :]
    print(mag_ang.shape)
    mag, ang = np.dsplit(mag_ang, 2)
    for i in range(9):
    #     print(x_stft[0, :, i].shape)
#         a = model.predict(np.dstack([  mag[0, :, i], ang[0, :, i]    ]))
        m1 = np.array([   mag[0, :, i:i+1]   ])
        a = np.dstack([       model_predict_mag(model, m1), ang[0, :, i]    ])
        print(denorm(a).shape)
        x_stft_pred.append(denorm(a).reshape(1025))

    x_stft_pred = np.array(x_stft_pred)
    x_stft_pred = np.swapaxes(x_stft_pred, 0, 1)
    
    np.save("x_stft_pred", x_stft_pred)

run_proc(test)

2022-09-15 19:43:01.756117: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-09-15 19:43:02.210690: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 2629 MB memory:  -> device: 0, name: GRID A100X-1-5C MIG 1g.5gb, pci bus id: 0000:8c:00.0, compute capability: 8.0
2022-09-15 19:43:09.671023: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2022-09-15 19:43:10.540951: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8101
2022-09-15 19:43:11.658420: I tensorflow/stream_executor/cuda/cuda_blas.cc:1760] TensorFloat-32 will be used for the matrix multipli

starting proc
(1, 1025, 18)
(1, 1025, 1)
(1, 1025, 1)
(1, 1025, 1)
(1, 1025, 1)
(1, 1025, 1)
(1, 1025, 1)
(1, 1025, 1)
(1, 1025, 1)
(1, 1025, 1)


In [106]:
#!g2.mig


def test():
    global x_stft_pred
    import numpy as np
    model = getAutoencoder()()
    model.load()
    
    x_stft_pred = []
    mag_ang = nnorm(get_x_stft(np.load("x.npy")))[:10, :, :]
    print(mag_ang.shape)
    mag, ang = np.dsplit(mag_ang, 2)
    
    m1 = mag[:, :, :]
    
    mag_pred = model_predict_mag(model, m1)
    ang_pred = ang[:, :, :]
    
    print(mag_pred.shape, ang_pred.shape)
    
    a = np.dstack((mag_pred, ang_pred))
    print(a.shape)
    x_stft_pred = denorm(a)
    print(x_stft_pred.shape)
#     x_stft_pred = .reshape(1025)

#     x_stft_pred = np.array(x_stft_pred)
#     x_stft_pred = np.swapaxes(x_stft_pred, 0, 1)
    
    np.save("x_stft_pred", x_stft_pred)

run_proc(test)

2022-09-15 20:06:53.001103: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-09-15 20:06:53.522106: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 2629 MB memory:  -> device: 0, name: GRID A100X-1-5C MIG 1g.5gb, pci bus id: 0000:8c:00.0, compute capability: 8.0
2022-09-15 20:07:00.892888: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2022-09-15 20:07:01.792924: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8101
2022-09-15 20:07:02.925166: I tensorflow/stream_executor/cuda/cuda_blas.cc:1760] TensorFloat-32 will be used for the matrix multipli

starting proc
(5, 1025, 18)
(5, 1025, 9) (5, 1025, 9)
(5, 1025, 18)
(5, 1025, 9)


In [117]:
#!g2.mig
import IPython.display as ipd
import librosa
import numpy as np
SAMPLE_RATE = 4096

ipd.Audio(data=librosa.istft(librosa.stft(np.load("x.npy")[4])), rate=SAMPLE_RATE)

In [116]:
#!g2.mig

ipd.Audio(data=librosa.istft(np.load("x_stft_pred.npy")[4]), rate=SAMPLE_RATE)

In [ ]:
#!g2.mig
plt.plot(np.abs(x_stft[0, :, 0]))
plt.show()
plt.plot(np.abs(x_stft_pred[:, 0]))

In [ ]:
#!g2.mig
import matplotlib.pyplot as plt

plt.plot(librosa.istft(librosa.stft(x[0])))
plt.plot(librosa.istft(x_stft_pred))


In [ ]:
#!g2.mig
librosa.istft(x_stft[0])

In [ ]:
#!g2.mig
librosa.istft(x_stft[0])

In [ ]:
#!g2.mig
librosa.istft(x_stft_pred)

In [ ]:
#!g2.mig
x_stft[0].shape

In [ ]:
#!g2.mig
x_stft_pred.shape

In [ ]:
#!g2.mig


In [ ]:
#!g2.mig
!nvidia-smi

In [ ]:
#!g2.mig
